In [1]:
# import os
# os.environ['HTTP_PROXY'] = "http://10.60.28.99:81"
# os.environ['HTTPS_PROXY'] = "http://10.60.28.99:81"

In [2]:
import nltk
import pickle
import sklearn_crfsuite
from nltk.corpus.reader import ConllCorpusReader
from sklearn_crfsuite import metrics

In [ ]:
# ! pip install nltk
# ! pip install sklearn_crfsuite
# ! pip install pandas

In [3]:
# ! ls CoNLL-2003
# ! head -20 "CoNLL-2003/train.txt"

! ls data/conll2003
! head -20 "data/conll2003/eng.train"

# Định dạng CONLL 
# Dữ liệu huấn luyện là những câu tiếng anh đã được gán nhãn NER: PER, LOC, ORG, MISC 
# "EU rejects German call ..."

eng.testa  eng.testb  eng.train
-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP B-NP B-LOC
1996-08-22 CD I-NP O

The DT B-NP O
European NNP I-NP B-ORG


In [4]:
import nltk
import os 

# nltk.download('punkt') # Simple tokenization
# nltk.download('averaged_perceptron_tagger') # English POS tagging

from nltk.corpus.reader import ConllCorpusReader
train_sents = ConllCorpusReader('data/conll2003', 'eng.train', ['words', 'pos', 'ignore', 'chunk']).iob_sents()
test_sents = ConllCorpusReader('data/conll2003', 'eng.testa', ['words', 'pos', 'ignore', 'chunk']).iob_sents()

In [5]:
# Bỏ các câu rỗng do lỗi dữ liệu
train_sents = [x for x in train_sents if len(x) > 0]
test_sents = [x for x in test_sents if len(x) > 0]

In [6]:
len(train_sents)

14041

In [7]:
train_sents[0]

[('EU', 'NNP', 'B-ORG'),
 ('rejects', 'VBZ', 'O'),
 ('German', 'JJ', 'B-MISC'),
 ('call', 'NN', 'O'),
 ('to', 'TO', 'O'),
 ('boycott', 'VB', 'O'),
 ('British', 'JJ', 'B-MISC'),
 ('lamb', 'NN', 'O'),
 ('.', '.', 'O')]

In [8]:
# Xem một số câu ví dụ 
train_sents[:3]

import pandas as pd
ll = []
for s in train_sents:
    for t in s: 
        ll.append({'token': t[0], 'postag': t[1], 'label': t[2]})
    ll.append({'token': '', 'postag': '', 'label': ''})
len(ll)    

df = pd.DataFrame(ll)
df.head(n=20)

,token,postag,label
0,EU,NNP,B-ORG
1,rejects,VBZ,O
2,German,JJ,B-MISC
3,call,NN,O
4,to,TO,O
5,boycott,VB,O
6,British,JJ,B-MISC
7,lamb,NN,O
8,.,.,O
9,,,


# Rút trích đặc trưng của từ, biểu diễn câu thành vector các đặc trưng

In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag, #VB, VBZ
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [10]:
# Xem kết quả của việc rút trích đặc trưng
sent2features(train_sents[0])


[{'bias': 1.0,
  'word.lower()': 'eu',
  'word[-3:]': 'EU',
  'word[-2:]': 'EU',
  'word.isupper()': True,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NNP',
  'postag[:2]': 'NN',
  'BOS': True,
  '+1:word.lower()': 'rejects',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VBZ',
  '+1:postag[:2]': 'VB'},
 {'bias': 1.0,
  'word.lower()': 'rejects',
  'word[-3:]': 'cts',
  'word[-2:]': 'ts',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'VBZ',
  'postag[:2]': 'VB',
  '-1:word.lower()': 'eu',
  '-1:word.istitle()': False,
  '-1:word.isupper()': True,
  '-1:postag': 'NNP',
  '-1:postag[:2]': 'NN',
  '+1:word.lower()': 'german',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'JJ',
  '+1:postag[:2]': 'JJ'},
 {'bias': 1.0,
  'word.lower()': 'german',
  'word[-3:]': 'man',
  'word[-2:]': 'an',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False

# Chuyển dataset thành dạng biểu diễn đặc trưng cho CRF


In [11]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]


CPU times: user 466 ms, sys: 86.8 ms, total: 552 ms
Wall time: 554 ms


In [12]:
print(len(X_train[0]))
print(len(y_train[0]))
print(X_train[0][2])
print(y_train[0][2])

9
9
{'bias': 1.0, 'word.lower()': 'german', 'word[-3:]': 'man', 'word[-2:]': 'an', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'JJ', 'postag[:2]': 'JJ', '-1:word.lower()': 'rejects', '-1:word.istitle()': False, '-1:word.isupper()': False, '-1:postag': 'VBZ', '-1:postag[:2]': 'VB', '+1:word.lower()': 'call', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'NN', '+1:postag[:2]': 'NN'}
B-MISC


# Huấn luyện CRF


In [13]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass





loading training data to CRFsuite: 100%|██████████| 14041/14041 [00:01<00:00, 8399.02it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 86687
Seconds required: 0.356

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.27  loss=232367.34 active=86384 feature_norm=1.00
Iter 2   time=0.14  loss=217022.74 active=83848 feature_norm=3.45
Iter 3   time=0.14  loss=161383.49 active=83844 feature_norm=2.99
Iter 4   time=0.41  loss=119552.24 active=83579 feature_norm=2.74
Iter 5   time=0.15  loss=94630.26 active=86129 feature_norm=3.20
Iter 6   time=0.14  loss=88330.83 active=85472 feature_norm=3.56
Iter 7   time=0.14  loss=71522.54 active=80105 feature_norm=5.12
Iter 8   time=0.14  loss=61055.57 active=64864 feature_norm=6.38
Iter 9   time=0.14  loss=54560.88 active=61779 feature_norm=7.80
Iter 10  t

# Đánh giá mô hình

In [14]:
X_test[0][1]

{'bias': 1.0,
 'word.lower()': '-',
 'word[-3:]': '-',
 'word[-2:]': '-',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': ':',
 'postag[:2]': ':',
 '-1:word.lower()': 'cricket',
 '-1:word.istitle()': False,
 '-1:word.isupper()': True,
 '-1:postag': 'NNP',
 '-1:postag[:2]': 'NN',
 '+1:word.lower()': 'leicestershire',
 '+1:word.istitle()': False,
 '+1:word.isupper()': True,
 '+1:postag': 'NNP',
 '+1:postag[:2]': 'NN'}

In [15]:
from itertools import chain


labels = list(crf.classes_)
print(labels)

y_pred = crf.predict(X_test)

print("predict of first example")
print(y_pred[:2])
print("gold")
print(y_test[:2])
yy_test = list(chain(*y_test))
yy_pred = list(chain(*y_pred))
print(len(yy_test))
print(len(yy_pred))

from sklearn import metrics
labels.remove('O')
print("All labels without O")
print(metrics.classification_report(yy_test, yy_pred, labels=labels, digits=3))  
print("All labels ")
print(metrics.classification_report(yy_test, yy_pred, digits=3))  
print(metrics.f1_score(yy_test, yy_pred, average='weighted', labels=['B-LOC', 'I-LOC']))
# score1 = metrics.flat_f1_score(, ))


# print(score1)

# score2 = metrics.flat_f1_score(y_test, y_pred,
#                       average='weighted', labels=labels)
# print(labels)
# print(score2)


['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']
predict of first example
[['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O']]
gold
[['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O']]
51362
51362
All labels without O
              precision    recall  f1-score   support

       B-ORG      0.851     0.809     0.830      1341
      B-MISC      0.926     0.839     0.881       922
       B-PER      0.901     0.908     0.905      1842
       I-PER      0.941     0.955     0.948      1307
       B-LOC      0.914     0.876     0.894      1837
       I-ORG      0.817     0.824     0.820       751
      I-MISC      0.885     0.737     0.804       346
       I-LOC      0.878     0.786     0.830       257

   micro avg      0.896     0.868     0.882      8603
   macro avg      0.889     0.842     0.864      8603
weighted avg      0.896     0.868     0.881      8603

All labels 
              precision    recall  f1-

In [16]:
for x, y in zip(y_pred, y_test):
    if len(x) != len(y):
        print(len(x), len(y))

In [17]:
len(list(chain(*y_test)))

51362

In [18]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)


from sklearn.metrics import classification_report
print(classification_report(
    list(chain(*y_test)), list(chain(*y_pred)), labels=sorted_labels, digits=3
))

# report = classification_report(y_test, y_pred, labels=sorted_labels, digits=3)



              precision    recall  f1-score   support

       B-LOC      0.914     0.876     0.894      1837
       I-LOC      0.878     0.786     0.830       257
      B-MISC      0.926     0.839     0.881       922
      I-MISC      0.885     0.737     0.804       346
       B-ORG      0.851     0.809     0.830      1341
       I-ORG      0.817     0.824     0.820       751
       B-PER      0.901     0.908     0.905      1842
       I-PER      0.941     0.955     0.948      1307

   micro avg      0.896     0.868     0.882      8603
   macro avg      0.889     0.842     0.864      8603
weighted avg      0.896     0.868     0.881      8603



In [19]:
# y_test, y_pred

In [20]:
for a, b in zip(test_sents[0], crf.predict([X_test[0]])[0]):
    print(a, b)


('CRICKET', 'NNP', 'O') O
('-', ':', 'O') O
('LEICESTERSHIRE', 'NNP', 'B-ORG') B-ORG
('TAKE', 'NNP', 'O') O
('OVER', 'IN', 'O') O
('AT', 'NNP', 'O') O
('TOP', 'NNP', 'O') O
('AFTER', 'NNP', 'O') O
('INNINGS', 'NNP', 'O') O
('VICTORY', 'NN', 'O') O
('.', '.', 'O') O


# Dự đoán nhãn NER cho một câu mới

In [21]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [22]:
import nltk
sentence = nltk.word_tokenize("He is a German who works at Facebook Inc.")
print("sentence", sentence)
x = nltk.pos_tag(sentence)
print("POS tag", x)
test_sent = [i + ('-', ) for i in x]
print("test_sent")
test_sent




sentence ['He', 'is', 'a', 'German', 'who', 'works', 'at', 'Facebook', 'Inc', '.']
POS tag [('He', 'PRP'), ('is', 'VBZ'), ('a', 'DT'), ('German', 'JJ'), ('who', 'WP'), ('works', 'VBZ'), ('at', 'IN'), ('Facebook', 'NNP'), ('Inc', 'NNP'), ('.', '.')]
test_sent


[('He', 'PRP', '-'),
 ('is', 'VBZ', '-'),
 ('a', 'DT', '-'),
 ('German', 'JJ', '-'),
 ('who', 'WP', '-'),
 ('works', 'VBZ', '-'),
 ('at', 'IN', '-'),
 ('Facebook', 'NNP', '-'),
 ('Inc', 'NNP', '-'),
 ('.', '.', '-')]

In [23]:
test_sent_feature = sent2features(test_sent)



In [24]:
# sentence ['He', 'is', 'a', 'German', 'who', 'works', 'at', 'Google', 'Inc', '.']
print(sentence)
print(len(sentence))
result = crf.predict([test_sent_feature])[0]
print(result)
print(len(result))
for a, b in zip(sentence, result):
    print(a, "==>", b)


['He', 'is', 'a', 'German', 'who', 'works', 'at', 'Facebook', 'Inc', '.']
10
['O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O']
10
He ==> O
is ==> O
a ==> O
German ==> B-MISC
who ==> O
works ==> O
at ==> O
Facebook ==> B-ORG
Inc ==> I-ORG
. ==> O


# Bài tập:
- Chuyển sang dạng CONLL từ dữ liệu gán nhãn `xml`
- Từ dạng IOB rút trích ra các NER
- Thực hiện Đánh giá theo từng loại entity
- Thử nghiệm với dữ liệu tiếng việt


```
<ENAMEX TYPE="LOCATION">Mỹ</ENAMEX> kêu gọi <ENAMEX TYPE="LOCATION">Nga</ENAMEX>, <ENAMEX TYPE="LOCATION">Trung</ENAMEX> dừng phổ biến vũ khí hạt nhân

Ngoại trưởng <ENAMEX TYPE="LOCATION">Mỹ</ENAMEX> <ENAMEX TYPE="PERSON">Rex Tillerson</ENAMEX> kêu gọi cộng đồng quốc tế ngăn chặn các nước sở hữu vũ khí hạt nhân.

Trong lời kêu gọi dừng các hoạt động tạo điều kiện để vũ khí hạt nhân phổ biến, Ngoại trưởng <ENAMEX TYPE="LOCATION">Mỹ</ENAMEX> <ENAMEX TYPE="PERSON">Rex Tillerson</ENAMEX> nêu đích danh <ENAMEX TYPE="LOCATION">Nga</ENAMEX>, <ENAMEX TYPE="LOCATION">Trung Quốc</ENAMEX> - ám chỉ các nước này nên gia tăng áp lực nhằm buộc <ENAMEX TYPE="LOCATION">Triều Tiên</ENAMEX> phải từ bỏ tham vọng vũ khí hạt nhân.
Ông <ENAMEX TYPE="PERSON">Tillerson</ENAMEX> lấy <ENAMEX TYPE="LOCATION">Triều Tiên</ENAMEX> làm bài học điển hình cho việc thất bại trong hoạt động ngăn chặn các nước sở hữu vũ khí tiêu diệt hàng loạt.
Ngoại trưởng <ENAMEX TYPE="LOCATION">Mỹ</ENAMEX> đưa ra phát ngôn trên trong cuộc họp cấp Bộ trưởng tại <ENAMEX TYPE="ORGANIZATION">Hội đồng Bảo an Liên Hợp Quốc</ENAMEX> do <ENAMEX TYPE="LOCATION">Mỹ</ENAMEX> kêu gọi tổ chức nhằm giải quyết mối đe doạ từ các hoạt động phát triển các vũ khí sinh học, hoá học, hạt nhân.
Ông <ENAMEX TYPE="PERSON">Tillerson</ENAMEX> nhắc lại, <ENAMEX TYPE="LOCATION">Washington</ENAMEX> và <ENAMEX TYPE="LOCATION">Moscow</ENAMEX> từng hợp tác rất tốt trong thời kỳ Xô-viết về các biện pháp giải trừ vũ khí, dù là đối thủ Chiến tranh Lạnh với nhau. Vậy nên, bây giờ “hai nước nên hợp tác như vậy một lần nữa”.
Hiện nay, thế giới đang nóng vấn đề vũ khí hạt nhân tên lửa khi <ENAMEX TYPE="LOCATION">Triều Tiên</ENAMEX> không ngừng thực hiện các vụ thử tên lửa, hạt nhân với sức mạnh không ngừng phát triển bất chấp các lệnh trừng phạt từ cộng đồng quốc tế


```


In [25]:
print("Done!")

Done!
